In [1]:
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import glob
import matplotlib.pyplot as plt
import tqdm
import pywt
import scipy.signal
import scipy.ndimage
import ecg
import tqdm
import joblib
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [ ]:
guohuan = pd.read_csv(r'D:\\byx\\baseline\\myEcg\\data\\过缓.csv')

In [ ]:
for index,age in zip(guohuan['index'],guohuan['age']):
    print(index,age)

## Test

In [ ]:
TP,FN = 0,0
error = []
for txtname,age in zip(guohuan['index'],guohuan['age']):
    df = pd.read_csv(r'D:\\byx\\baseline\\myEcg\\data\\hf_round2_train\\%s'%txtname,delimiter=' ')
    sampling_rate = 500
    if ecg.raise_error(df['I']):
        print('error',txtname)
        continue
    #peaks = R_peak(df,plot=False)
    _,_,epochs = ecg.segment(df,plot=False)
    #print((np.mean(r['T_Onsets']) - np.mean(r['P_Offsets'])) / 500)
    #print('心率：',60 / len(epochs[0]) * sampling_rate)
    xinlv = 60 / len(epochs[0]) * sampling_rate
    if xinlv > 60:
        #print(txtname)
        #peaks = ecg.R_peak(df,plot=True)
        #error.append(txtname)
        FN += 1
    else:
        TP += 1
print(TP/(TP+FN))

In [ ]:
FP,TN = 0,0
trains = glob.glob(r'D:\\byx\\baseline\\myEcg\\data\\hf_round2_train\\**.txt')
c = 0
for txtname in tqdm.tqdm(trains[:100]):
    txtn = txtname.split('\\')[-1]
    if txtn not in guohuan['index']:
        df = pd.read_csv(txtname,delimiter=' ')
        if ecg.raise_error(df['I']):
            print('error',txtname)
            continue
        _,_,epochs,_ = ecg.segment(df,plot=False)
        xinlv = 60 / len(epochs[0]) * sampling_rate
        if xinlv > 60:
            #print(txtname)
            #peaks = ecg.R_peak(df,plot=True)
            #error.append(txtname)
            TN += 1
        else:
            FP += 1
            peaks = ecg.R_peak(df,plot=True)
            print(xinlv)

In [ ]:
Precision = TP/(TP+FP)
Recall = TP/(TP+FN)
F1 = 2 * (Precision*Recall)/(Precision+Recall)
Accuracy = (TP+TN) / (TP+FP+FN+TN)
print(f'Precision = {Precision}')
print(f'Recall = {Recall}')
print(f'F1 = {F1}')
print(f'Accuracy = {Accuracy}')

# building dataset

In [ ]:
def heart_rating(txtname,sampling_rate=500):
    df = pd.read_csv(r'D:\\byx\\baseline\\myEcg\\data\\hf_round2_train\\%s'%txtname,delimiter=' ')
    if ecg.raise_error(df['I']):
        print('error',txtname)
        return np.nan
    _,_,epochs,_ = ecg.segment(df,plot=False)
    xinlv = 60 / len(epochs[0]) * sampling_rate
    return xinlv

In [ ]:
dataset = []
root = 'D:\\byx\\baseline\\myEcg\\data'
train_csv = os.path.join(root,'hf_round2_label_train.csv')
train = pd.read_csv(train_csv)
train = train.dropna(axis=0,how='any')  

In [ ]:
train['one_label'] = train['one_label'].map(lambda x: eval(x))
train['guohuan'] = train['one_label'].map(lambda x:1 if x[18]==1.0 else 0)
train['heart_ratings'] = train['index'].map(lambda x:heart_rating(x))
train = train.dropna(axis=0,how='any')  

In [ ]:
train['sex2'] = train['sex'].map(lambda x:1 if x=='MALE' else 0)
train_size = int(len(train)*0.5)
train_dataset = train[:train_size]
test_dataset = train[train_size:]
X_train = train_dataset[['age','heart_ratings','sex2']]
y_train = train_dataset['guohuan']
X_test = test_dataset[['age','heart_ratings','sex2']]
y_test = test_dataset['guohuan']

In [ ]:
train.to_csv('train_heartrating.csv')

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pre = clf.predict(X_test)
recall = recall_score(y_test,y_pre)
accuracy = accuracy_score(y_test,y_pre)
precision = precision_score(y_test,y_pre)
F1 = f1_score(y_test,y_pre)
print(f'Precision = {precision}')
print(f'*Recall* = {recall}')
print(f'F1 = {F1}')
print(f'Accuracy = {accuracy}')

In [ ]:
train[train['guohuan']==1]

In [ ]:
clf.coef_,clf.intercept_

# 过速

In [ ]:
train['guosu'] = train['one_label'].map(lambda x:1 if x[30]==1.0 else 0)

In [ ]:
train_size = int(len(train)*0.5)
train_dataset = train[:train_size]
test_dataset = train[train_size:]
X_train = train_dataset[['age','heart_ratings']]
y_train = train_dataset['guosu']
X_test = test_dataset[['age','heart_ratings']]
y_test = test_dataset['guosu']

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pre = clf.predict(X_test)
recall = recall_score(y_test,y_pre)
accuracy = accuracy_score(y_test,y_pre)
precision = precision_score(y_test,y_pre)
F1 = f1_score(y_test,y_pre)
print(f'Precision = {precision}')
print(f'*Recall* = {recall}')
print(f'F1 = {F1}')
print(f'Accuracy = {accuracy}')

### 过速 - 东阳数据集

In [5]:
def heart_rating(txtname,sampling_rate=500):
    df = pd.read_csv(r'D:\\byx\\Dongyang\\txt_data\\%s'%txtname,delimiter=' ')
    if ecg.raise_error(df['I']) or ecg.raise_error(df['II']):
        print('error',txtname)
        return np.nan
    try:
        peaks = ecg.R_peak(df,lead='II')
        xinlv = 60/((peaks[-1]-peaks[0])/(len(peaks)-1)/500)
    except Exception as e:
        print('***',txtname)
        return np.nan
    return xinlv

In [6]:
train = pd.read_csv(r'D:\byx\Dongyang\label_add_heartrating_2.csv')
train['heart_rating_updated'] = train['0'].map(lambda x:heart_rating(x))

error FILE@1e0b8bbcdef629bcbd9c9b2a1ec089fe.txt
error FILE@4f1a221eb1c6dfb2c60cc1cde87d4e14.txt
error FILE@d15c379778cf7350a4a176d5808d8610.txt
error FILE@e737727ffd67ec307c9e3215c58363a2.txt


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  


error FILE@ad05a8eae444a886d170e0afe625e220.txt
*** FILE@3fad165d7f5d570245b99c0404f06320.txt
error FILE@bdd1c1b39973d2acbc5e6369002aa41a.txt
error FILE@23c8361eb7b9ab521a1a426fca418653.txt
var 0.868702916605499
error FILE@155676cebef1303fce8d38e997490d8b.txt
error FILE@4b250ee5e9354bc2bd7d3cbcc5ad6baf.txt
error FILE@0c0f9b0f9d64edbe45a41b68fbc4eb48.txt
*** FILE@e17105813b0b68f28f9302b1084ef27f.txt
error FILE@ebaacace99c24fe98b37706b46e6bf06.txt
error FILE@20307972eb1ce087834b3b22cafb13fd.txt
error FILE@0f0b2b8a774ad53eede2d153a1b8b8b4.txt
error FILE@121495d6143ee2b243d4411f4b580e1c.txt
*** FILE@0c1f0f410a784f2d827237925e50613d.txt
error FILE@8c3ae29d1b4094b3179f0166be65104c.txt
error FILE@0a50273cf10b9aed5362e3bfdc1a63f4.txt
var 0.9402701742497822
error FILE@2edb24ef539f1764818fa3e704b50ede.txt
error FILE@b053cc4414c5b20f59f33c9ce0828a2b.txt
error FILE@87988a6ceed5bedf2afbc61af3479ca8.txt
error FILE@a0661d50fd9b9fc485288763a7178f0c.txt
error FILE@ae0d004424de6acc8f344ab1ab252629.txt
*

In [ ]:
train = train.dropna(axis=0,how='any')  
train_size = int(len(train)*0.5)
train_dataset = train[:train_size]
test_dataset = train[train_size:]
X_train = train_dataset[['heart_rating']]
y_train = train_dataset['过缓']
X_test = test_dataset[['heart_rating']]
y_test = test_dataset['过缓']

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pre = clf.predict(X_test)
recall = recall_score(y_test,y_pre)
accuracy = accuracy_score(y_test,y_pre)
precision = precision_score(y_test,y_pre)
F1 = f1_score(y_test,y_pre)
print(f'Precision = {precision}')
print(f'*Recall* = {recall}')
print(f'F1 = {F1}')
print(f'Accuracy = {accuracy}')

In [ ]:
y_pre = train['heart_rating'].map(lambda x:1 if x < 60 else 0)
y_test = train['过缓']
recall = recall_score(y_test,y_pre)
accuracy = accuracy_score(y_test,y_pre)
precision = precision_score(y_test,y_pre)
F1 = f1_score(y_test,y_pre)
print(f'Precision = {precision}')
print(f'*Recall* = {recall}')
print(f'F1 = {F1}')
print(f'Accuracy = {accuracy}')

In [ ]:
train[train['过缓']==1]